In [1]:
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.lib import FractionalBacktest
import tqdm

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def SMA(values, n):
    return pd.Series(values).rolling(n).mean()

class SMACross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma_1 = self.I(SMA, self.data.Close, self.n1)
        self.sma_2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma_1 cross above sma_2, close any position
        # Short trade and buy asset
        if crossover(self.sma_1, self.sma_2):
            self.position.close()
            self.buy(size=0.95)
        # ELse sma_2 cross above sma_1, close any position
        # Long trade and sell asset
        elif crossover(self.sma_2, self.sma_1):
            self.position.close()
            self.sell(size=0.95)

In [3]:
# Download data
df = yf.download('BTC-USD', auto_adjust=True)
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
df.index = df.index.tz_localize(None)

# Run backtest
bt = FractionalBacktest(
    df,
    SMACross,
    cash=100_000,
    commission=0.002,
    trade_on_close=True,
    finalize_trades=True
)

stats = bt.run()
print(stats)

bt.plot()

[*********************100%***********************]  1 of 1 completed


Start                     2025-12-27 00:00:00
End                       2026-01-27 00:00:00
Duration                     31 days 00:00:00
Exposure Time [%]                        6.25
Equity Final [$]                 102326.93256
Equity Peak [$]                  102511.15178
Commissions [$]                     373.83997
Return [%]                            2.32693
Buy & Hold Return [%]                -8.94475
Return (Ann.) [%]                    30.00137
Volatility (Ann.) [%]                11.26998
CAGR [%]                             31.10633
Sharpe Ratio                          2.66206
Sortino Ratio                        48.65323
Calmar Ratio                        166.94649
Alpha [%]                             1.55575
Beta                                 -0.08622
Max. Drawdown [%]                    -0.17971
Avg. Drawdown [%]                    -0.17971
Max. Drawdown Duration                 0.0018
Avg. Drawdown Duration                 0.0018
# Trades                          

GridPlot(id='p1396', ...)

In [4]:
print(stats['_trades'].to_string())

       Size  EntryBar  ExitBar    EntryPrice     ExitPrice   SL   TP          PnL  Commission  ReturnPct  EntryTime   ExitTime Duration   Tag  Entry_SMA(C,10)  Exit_SMA(C,10)  Entry_SMA(C,20)  Exit_SMA(C,20)
0 -1.064033        29       30  89104.765625  86566.523438  NaN  NaN  2326.932565  373.839971   0.024543 2026-01-25 2026-01-26   1 days  None         0.000918        0.000909         0.000922        0.000918


In [5]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 50, 5), 'n2': range(10, 100, 5)}

# Run the optimization
res = bt.optimize(**param_grid)

print('Best result: ', res['Return [%]'])
print('Parameters for best result: ', res['_strategy'])

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
Backtest.optimize:   0%|          | 0/162 [00:00<?, ?it/s]
































































































































































































































































































































































Best result:  -13.662844786779823
Parameters for best result:  SMACross(n1=5,n2=10)


In [6]:
res

Start                     2025-12-27 00:00:00
End                       2026-01-27 00:00:00
Duration                     31 days 00:00:00
Exposure Time [%]                      46.875
Equity Final [$]                  86337.15521
Equity Peak [$]                      100000.0
Commissions [$]                    1049.20913
Return [%]                          -13.66284
Buy & Hold Return [%]                -3.44977
Return (Ann.) [%]                   -81.28217
Volatility (Ann.) [%]                 4.75783
CAGR [%]                             -82.2671
Sharpe Ratio                        -17.08389
Sortino Ratio                        -3.35298
Calmar Ratio                         -5.02099
Alpha [%]                           -13.88384
Beta                                 -0.06406
Max. Drawdown [%]                   -16.18846
Avg. Drawdown [%]                   -16.18846
Max. Drawdown Duration       15 days 00:00:00
Avg. Drawdown Duration       15 days 00:00:00
# Trades                          

In [7]:
bt.plot()

GridPlot(id='p1774', ...)